In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import urllib.parse
import re
import numpy as np
import seaborn as sns
import ast
from wordcloud import WordCloud, STOPWORDS
import sklearn
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import resample
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_selection import SelectFromModel
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_1 = pd.read_parquet('/Users/lijiazheng/Desktop/success_model_final.parquet')
df_1

,cls_embedding,trial_label,identificationModule_nctId,eligibilityModule_minimumAge,eligibilityModule_maximumAge,designModule_enrollmentInfo_count,PHASE1,PHASE2,PHASE3,PHASE4,...,Inorganic Chemicals,Enzymes and Coenzymes,Carbohydrates,Biological Factors,Pharmaceutical Preparations,"Nucleic Acids, Nucleotides, and Nucleosides",Complex Mixtures,Macromolecular Substances,Biomedical and Dental Materials,combined_description
0,"[0.203131, 0.052898195, 0.1826738, -0.11610553...",Positive,NCT01817452,18.0,100.0,220.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trial to evaluate efficacy of dual blockade wi...
1,"[0.067105345, 0.38064072, 0.4827489, 0.3155071...",Positive,NCT01772472,18.0,100.0,1487.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"This 2-arm, randomized, open-label study will ..."
2,"[-0.3251111, 0.031221235, 0.053585365, 0.57446...",Positive,NCT00157703,19.0,100.0,9.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"This is an open-label, single site study to ev..."
3,"[0.026379747, 0.2683024, 0.3200358, 0.31718358...",Positive,NCT05428566,18.0,35.0,144.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,The main objective of the study is to evaluate...
4,"[-0.19727723, -0.15298994, 0.10479475, 0.94340...",Negative,NCT02250599,18.0,100.0,80.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"The present study will be a randomized, contro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11772,"[0.12481902, 0.17882024, 0.13729087, -0.052308...",Negative,NCT00300885,18.0,100.0,926.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A randomized controlled trial comparing safety...
11773,"[-0.058878843, -0.016227359, 0.30076474, 0.087...",Positive,NCT00010452,0.0,18.0,150.0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,OBJECTIVES:\n\nI. Determine the efficacy of pi...
11774,"[0.15578431, 0.3220898, 0.122477956, 0.642282,...",Positive,NCT01217216,30.0,75.0,52.0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,To test the effectiveness and feasibility of i...
11775,"[-0.14418244, -0.06669695, 0.19938894, 0.42481...",Positive,NCT00908180,16.0,65.0,47.0,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,RATIONALE: Giving chemotherapy before a donor ...


In [ ]:
df_2 = pd.read_parquet('/Users/lijiazheng/Desktop/trials_pmid_papers_results.parquet')

In [13]:
df_flag = df_2[['protocolSection.identificationModule.nctId', 'has_drug_or_bio']].copy()

# Step 2: Merge the flag into df_1 based on NCT ID
df_1 = df_1.merge(
    df_flag,
    how='left',
    left_on='identificationModule_nctId',
    right_on='protocolSection.identificationModule.nctId'
)

# Step 3: Keep only rows where has_drug_or_bio is True
df_1_filtered = df_1[df_1['has_drug_or_bio'] == True].copy()
df_1_filtered 

,cls_embedding,trial_label,identificationModule_nctId,eligibilityModule_minimumAge,eligibilityModule_maximumAge,designModule_enrollmentInfo_count,PHASE1,PHASE2,PHASE3,PHASE4,...,Carbohydrates,Biological Factors,Pharmaceutical Preparations,"Nucleic Acids, Nucleotides, and Nucleosides",Complex Mixtures,Macromolecular Substances,Biomedical and Dental Materials,combined_description,protocolSection.identificationModule.nctId,has_drug_or_bio
0,"[0.203131, 0.052898195, 0.1826738, -0.11610553...",Positive,NCT01817452,18.0,100.0,220.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trial to evaluate efficacy of dual blockade wi...,NCT01817452,True
1,"[0.067105345, 0.38064072, 0.4827489, 0.3155071...",Positive,NCT01772472,18.0,100.0,1487.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"This 2-arm, randomized, open-label study will ...",NCT01772472,True
2,"[-0.3251111, 0.031221235, 0.053585365, 0.57446...",Positive,NCT00157703,19.0,100.0,9.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"This is an open-label, single site study to ev...",NCT00157703,True
4,"[-0.19727723, -0.15298994, 0.10479475, 0.94340...",Negative,NCT02250599,18.0,100.0,80.0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"The present study will be a randomized, contro...",NCT02250599,True
5,"[-0.45779037, -0.08309143, 0.027704647, 0.7263...",Positive,NCT00711243,18.0,100.0,59.0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"RATIONALE: Drugs used in chemotherapy, such as...",NCT00711243,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11770,"[0.25984997, -0.03634331, 0.20794162, 0.234410...",Positive,NCT04764643,18.0,80.0,140.0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Lugol chromoendoscopy is essential in detectin...,NCT04764643,True
11772,"[0.12481902, 0.17882024, 0.13729087, -0.052308...",Negative,NCT00300885,18.0,100.0,926.0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,A randomized controlled trial comparing safety...,NCT00300885,True
11773,"[-0.058878843, -0.016227359, 0.30076474, 0.087...",Positive,NCT00010452,0.0,18.0,150.0,0,1,1,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,OBJECTIVES:\n\nI. Determine the efficacy of pi...,NCT00010452,True
11775,"[-0.14418244, -0.06669695, 0.19938894, 0.42481...",Positive,NCT00908180,16.0,65.0,47.0,0,1,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,RATIONALE: Giving chemotherapy before a donor ...,NCT00908180,True


In [14]:
df_1 = df_1_filtered.reset_index(drop=True)

In [ ]:
df_1['status_binary'] = (df_1['trial_label'] == 'Positive').astype(int)
print(df_1['status_binary'].value_counts())

df_1 = df_1.drop(columns = ['identificationModule_nctId','trial_label', 'combined_description'])
df_1 = df_1 = df_1.drop(columns = ['protocolSection.identificationModule.nctId','has_drug_or_bio'])

status_binary
1    5707
0    2656
Name: count, dtype: int64


In [21]:
# === CONFIG ===
n_runs = 2
desired_ratio = 5707/2656
param_grid_1 = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5 ],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}
param_grid_2 = {
    'n_estimators': [300, 350, 400],
    'max_depth': [3, 4, 5 ],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}
# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df_1.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.05, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, param_grid,feature_names=None, return_importance=False):
        clf = xgb.XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            use_label_encoder=False,
            scale_pos_weight=2656/5707
        )
        grid = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        # y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob > 0.4).astype(int)

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test, param_grid_1,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb, param_grid_2) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===
print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))


===================== Run 1/2 =====================
Optimal PCA components: 273


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 150, 'subsample': 0.8}
Best parameters for this run: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 400, 'subsample': 0.8}

===================== Run 2/2 =====================
Optimal PCA components: 273


/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/Users/lijiazheng/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Best parameters for this run: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.7}
Best parameters for this run: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 400, 'subsample': 0.7}

===================== Top Features Without Embeddings =====================
designInfo_allocation_RANDOMIZED: 0.0777
PHASE1: 0.0580
interventionModel_SINGLE_GROUP: 0.0391
leadsponsor_NETWORK: 0.0350
organization_class_NIH: 0.0326
Immune System Diseases: 0.0300
organization_class_INDUSTRY: 0.0279
leadsponsor_INDUSTRY: 0.0269
FdaRegulatedDevice_no: 0.0228
masking_SINGLE: 0.0227
PHASE2: 0.0219
collaborator_NETWORK: 0.0216
Cardiovascular Diseases: 0.0211
eligibilityModule_maximumAge: 0.0207
FdaRegulatedDrug_no: 0.0202
collaborator_NIH: 0.0202
Infections: 0.0193
facility_count: 0.0192
primaryPurpose_DIAGNOSTIC: 0.0190
interventionModel_CROSSOVER: 0.0189

===== Aggregated Classification Report (No Embeddings) =====
  class    p

In [22]:
results

defaultdict(list,
            {'no_emb_feature_importances': [{'eligibilityModule_minimumAge': 0.014420013,
               'eligibilityModule_maximumAge': 0.020558108,
               'designModule_enrollmentInfo_count': 0.017224701,
               'PHASE1': 0.062082455,
               'PHASE2': 0.024966918,
               'PHASE3': 0.017599715,
               'PHASE4': 0.016187944,
               'organization_class_INDUSTRY': 0.02720801,
               'organization_class_NETWORK': 0.018648408,
               'organization_class_NIH': 0.031218365,
               'organization_class_OTHER_GOV': 0.018096369,
               'oversightHasDmc_no': 0.013731854,
               'FdaRegulatedDrug_no': 0.019183332,
               'FdaRegulatedDevice_no': 0.024646329,
               'designInfo_allocation_NON_RANDOMIZED': 0.015611869,
               'designInfo_allocation_RANDOMIZED': 0.065913886,
               'interventionModel_SINGLE_GROUP': 0.03828928,
               'primaryPurpose_SUPPORT

In [ ]:
# === CONFIG ===
n_runs = 5
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20],
    'max_features': ['sqrt', 'log2']
}

# === RESULTS STORAGE ===
results = defaultdict(list)

# === LOOP ===
for run in range(n_runs):
    print(f"\n===================== Run {run+1}/{n_runs} =====================")

    # === Step 1: Prepare Dataset ===
    df_run = df_1.copy()
    # Undersample majority class
    df_minority = df_run[df_run['status_binary'] == 0]
    df_majority = df_run[df_run['status_binary'] == 1].sample(
        n=int(desired_ratio * len(df_minority)), random_state=run
    )
    df_sampled = pd.concat([df_majority, df_minority]).sample(frac=1, random_state=run).reset_index(drop=True)

    # === Step 2: PCA on cls_embedding ===
    embeddings_matrix = np.vstack(df_sampled['cls_embedding'].values)
    embeddings_matrix = StandardScaler().fit_transform(embeddings_matrix)  # Normalize before PCA

    pca = PCA()
    pca.fit(embeddings_matrix)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    optimal_components = np.argmax(cumulative_variance >= 0.95) + 1
    print(f"Optimal PCA components: {optimal_components}")

    pca = PCA(n_components=optimal_components)
    reduced_embeddings = pca.fit_transform(embeddings_matrix)
    reduced_embeddings = StandardScaler().fit_transform(reduced_embeddings)

    df_pca = pd.DataFrame(reduced_embeddings, columns=[f'pca_emb_{i}' for i in range(optimal_components)])
    df_pca.index = df_sampled.index

    # === Step 3: Feature Processing ===
    df_sampled = df_sampled.drop(columns=['cls_embedding'])  # Drop embedding
    X = df_sampled.drop(columns=['status_binary'])
    y = df_sampled['status_binary']

    # Identify feature types
    numerical_features = [col for col in X.columns if X[col].dtype in [np.float64, np.int64]]
    dummy_features = [col for col in X.columns if col not in numerical_features]

    # Convert boolean to int
    for col in dummy_features:
        if X[col].dtype == 'bool':
            X[col] = X[col].astype(int)

    # Create X_emb
    X_emb = pd.concat([X, df_pca], axis=1)

    # Standardize tabular and embedding features
    scaler = StandardScaler()
    X[numerical_features] = scaler.fit_transform(X[numerical_features])
    X_emb[numerical_features] = scaler.transform(X_emb[numerical_features])
    X_emb[df_pca.columns] = scaler.fit_transform(X_emb[df_pca.columns])

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
    X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(X_emb, y, stratify=y, test_size=0.2, random_state=42)

    # === Step 4: Feature Selection ===
    def apply_feature_selection(X_train, y_train, X_test):
        selector_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold='mean', prefit=True)

        selected_mask = selector.get_support()
        selected_feature_names = X_train.columns[selected_mask]  # <-- get the names

        return selector.transform(X_train), selector.transform(X_test), selected_feature_names

    X_train_sel, X_test_sel, selected_feature_names = apply_feature_selection(X_train, y_train, X_test)
    X_train_emb_sel, X_test_emb_sel, selected_feature_names_emb = apply_feature_selection(X_train_emb, y_train_emb, X_test_emb)

    def train_and_eval_all(X_train, y_train, X_test, y_test, feature_names=None, return_importance=False):
        clf = RandomForestClassifier(random_state=42, class_weight='balanced')
        grid = GridSearchCV(clf, param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)

        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best parameters for this run: {grid.best_params_}")
        y_pred = best_model.predict(X_test)
        y_prob = best_model.predict_proba(X_test)[:, 1]

        auc = roc_auc_score(y_test, y_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        # <-- Add classification report here
        report = classification_report(y_test, y_pred, output_dict=True)

        result = {
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'accuracy': accuracy,
            'f1': f1,
            'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'report': report  
        }

        if return_importance and feature_names is not None:
            importances = best_model.feature_importances_
            feature_importance_dict = dict(zip(feature_names, importances))
            result['feature_importance'] = feature_importance_dict

        return result

    # === Train and Evaluate ===
    # Pass selected feature names after feature selection
    metrics_no_emb = train_and_eval_all(
        X_train_sel, y_train, X_test_sel, y_test,
        feature_names=selected_feature_names,
        return_importance=True
    )

    metrics_with_emb = train_and_eval_all(
        X_train_emb_sel, y_train_emb, X_test_emb_sel, y_test_emb) # Don't return importance for embeddings
    results['no_emb_feature_importances'].append(metrics_no_emb['feature_importance'])

    # Save metrics to results
    for metric, value in metrics_no_emb.items():
        results[f'no_emb_{metric}'].append(value)
    for metric, value in metrics_with_emb.items():
        results[f'with_emb_{metric}'].append(value)
    
    results['no_emb_reports'].append(metrics_no_emb['report'])
    results['with_emb_reports'].append(metrics_with_emb['report'])


# Aggregate feature importances
importance_accumulator = defaultdict(list)

for run_dict in results['no_emb_feature_importances']:
    for feature, importance in run_dict.items():
        importance_accumulator[feature].append(importance)

# Compute mean importance for each feature
mean_importances = {feat: np.mean(imps) for feat, imps in importance_accumulator.items()}

# Sort and display top features
top_features = sorted(mean_importances.items(), key=lambda x: x[1], reverse=True)

print("\n===================== Top Features Without Embeddings =====================")
for feat, importance in top_features[:20]:  # Change 20 to whatever top-k you want
    print(f"{feat}: {importance:.4f}")

# === Step 5: Report Mean ± Std for all metrics ===
def aggregate_classification_reports_pretty(report_list, label_names=['0', '1']):
    metrics = ['precision', 'recall', 'f1-score', 'support']
    df_rows = []

    for label in label_names:
        row = {'class': label}
        for metric in metrics:
            values = [r[label][metric] for r in report_list]
            row[metric] = f"{np.mean(values):.2f} ± {np.std(values):.2f}"
        df_rows.append(row)
    
    return pd.DataFrame(df_rows)

# === After your loop ===

print("\n===== Aggregated Classification Report (No Embeddings) =====")
print(aggregate_classification_reports_pretty(results['no_emb_reports']))

print("\n===== Aggregated Classification Report (With Embeddings) =====")
print(aggregate_classification_reports_pretty(results['with_emb_reports']))

In [ ]:
results